<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Modern%20Convolutional%20Networks/networks_with_parallel_concatenations_GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
w_dir = '/content/drive/My Drive/Colab/D2L.AI/'
sys.path.append(w_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing Libraries

In [2]:
!pip uninstall mxnet && pip install mxnet-cu100

Importing Libraries

In [0]:
from mxnet import gluon, init, nd
from mxnet.gluon import nn
import d2l

Inception Blocks

In [0]:
class Inception(nn.Block):
  def __init__(self, c1, c2, c3, c4, **kwargs):
    super(Inception, self).__init__(**kwargs)
    # Path 1 is a single 1 x 1 convolutional layer
    self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
    # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3 convolutional layer
    self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
    self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1, activation='relu')
    # Path 3 is a 1 x 1 convolutional layer followed by a 5 x 5 convolutional layer
    self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
    self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2, activation='relu')
    # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1 convolutional layer
    self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
    self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')
    
  def forward(self, x):
    p1 = self.p1_1(x)
    p2 = self.p2_2(self.p2_1(x))
    p3 = self.p3_2(self.p3_1(x))
    p4 = self.p4_2(self.p4_1(x))
    # Concatenate the outputs on the channel dimension
    return nd.concat(p1, p2, p3, p4, dim=1)
    

GoogLeNet Model

In [0]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
       nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b3 = nn.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32),
       Inception(128, (128, 192), (32, 96), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b4 = nn.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64),
       Inception(160, (112, 224), (24, 64), 64),
       Inception(128, (128, 256), (24, 64), 64),
       Inception(112, (144, 288), (32, 64), 64),
       Inception(256, (160, 320), (32, 128), 128),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

In [0]:
b5 = nn.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128),
       Inception(384, (192, 384), (48, 128), 128),
       nn.GlobalAvgPool2D())

In [0]:
net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

In [11]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t', X.shape)

sequential0 output shape:	 (1, 64, 24, 24)
sequential1 output shape:	 (1, 192, 12, 12)
sequential2 output shape:	 (1, 480, 6, 6)
sequential3 output shape:	 (1, 832, 3, 3)
sequential4 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


Training

In [12]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on gpu(0)
epoch 1, loss 2.2696, train acc 0.150, test acc 0.175, time 60.5 sec
epoch 2, loss 1.2481, train acc 0.501, test acc 0.673, time 56.5 sec
epoch 3, loss 0.6427, train acc 0.756, test acc 0.796, time 57.2 sec
epoch 4, loss 0.4601, train acc 0.826, test acc 0.840, time 56.3 sec
epoch 5, loss 0.3825, train acc 0.856, test acc 0.869, time 56.6 sec
